**Bibliotecas utilizadas**

In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

**Criando Dataframe - COVID, FLU, COLD Symptoms**

In [75]:
URL = 'https://janeawsdata.s3.us-east-2.amazonaws.com/large_data.csv'
df = pd.read_csv(URL)

**Mapeando as classes para valores numéricos**


In [76]:
mappings = {
   'ALLERGY': 0,
   'COLD': 1,
   'COVID': 2,
   'FLU': 3
}
df['TYPE'] = df['TYPE'].apply(lambda x: mappings[x])

**Utilizando a função train_test_split para dividir os dados em 80/20 % treino e teste**

In [77]:
X = df.drop('TYPE', axis=1).values
y = df['TYPE'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Convertendo os dados para PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


**Definindo a Rede Neural**

In [78]:
class ANN(nn.Module):
   def __init__(self):
       super().__init__()
       # 2 camadas escondidas, número de neurônios das camadas foi escolhido de forma arbitrária
       self.fc1 = nn.Linear(in_features=20, out_features=40) #Entrada = 20 (features em x)
       self.fc2 = nn.Linear(in_features=40, out_features=20)
       self.output = nn.Linear(in_features=20, out_features=4) # Saída = 4 (classes)
 
   # função forward que será responsável pela propagação da rede
   def forward(self, x):
     x = F.relu(self.fc1(x)) # função relu para retornar sempre valores positivos
     x = F.relu(self.fc2(x))
     x = self.output(x)
     return x

**Instânciando o modelo**

In [79]:
model = ANN()

In [80]:
model.output

Linear(in_features=20, out_features=4, bias=True)

**Critério - CrossEntropyLoss e Algoritmo de otimização - Adam**



In [81]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) #taxa de aprendizagem de 0.01

**Treinamento de modelo - 100 épocas**

In [82]:
epochs = 100 
loss_arr = []
for i in range(epochs):
   y_hat = model.forward(X_train)
   loss = criterion(y_hat, y_train)
   loss_arr.append(loss)
 
   if i % 10 == 0:
       print(f'Epocas: {i} Perda: {loss}')
 
   optimizer.zero_grad()
   loss.backward()
   optimizer.step()

Epocas: 0 Perda: 1.4522038698196411
Epocas: 10 Perda: 0.4899253249168396
Epocas: 20 Perda: 0.2827320694923401
Epocas: 30 Perda: 0.2286471277475357
Epocas: 40 Perda: 0.19656157493591309
Epocas: 50 Perda: 0.17320752143859863
Epocas: 60 Perda: 0.15374161303043365
Epocas: 70 Perda: 0.13416050374507904
Epocas: 80 Perda: 0.11596114188432693
Epocas: 90 Perda: 0.10626749694347382


**Avaliando o modelo**

In [83]:
preds = []
with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

In [84]:
dfcorrecao = pd.DataFrame({'Y': y_test, 'YHat': preds})
dfcorrecao['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(dfcorrecao['Y'], dfcorrecao['YHat'])] #compara o valor de Y com o YHat valor estimado

In [85]:
print(classification_report(y_test, preds, df.TYPE.unique()))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3287
           1       0.54      0.66      0.59       210
           2       0.48      0.34      0.40       395
           3       0.94      0.96      0.95      4999

    accuracy                           0.93      8891
   macro avg       0.74      0.73      0.73      8891
weighted avg       0.93      0.93      0.93      8891

